# Block Viewing Demo
stough 202-

**Block viewing** is mportant for understanding ubiquitous image and video compression schemes like JPEG

Here we consider an image as a grid of non-overlapping blocks, each block containing a number of pixels. A **block transform** is where we replace each block from the original image with a block of our choosing. We could choose to replace the whole collection of pixels in a block with the average or median color of the block. 

The compression schemes we will be soon learning about rely on spatial coherence to consider each block as a collection of characteristics (mean, differences, whatever) rather than as a collection if independent pixels. Depending on the descriptive power of any particular characteristic, we may choose to heavily quantize or even ignore that characteristic in order to save space, leading to a lossy compression that does not badly affect the perceived image quality.

## Imports
We add several functions from the [`skimage`](https://scikit-image.org/) module:

- [`view_as_blocks`](https://scikit-image.org/docs/dev/api/skimage.util.html?highlight=view_as_blocks#skimage.util.view_as_blocks) decomoposes an image into blocks of a defined size.
- [`montage`](https://scikit-image.org/docs/dev/api/skimage.util.html?highlight=view_as_blocks#skimage.util.montage) allows us to reassemble a bunch of blocks back into a grid.
- We use [`resize`](https://scikit-image.org/docs/dev/api/skimage.transform.html?highlight=resize#skimage.transform.resize) or [`rescale`](https://scikit-image.org/docs/dev/api/skimage.transform.html?highlight=resize#skimage.transform.rescale) to ensure that our image and glyphs have a common denominator that we can use as a block size.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

import skimage.color as color

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair)

from skimage.util import view_as_blocks
from skimage.util import montage
from skimage.transform import resize, rescale

## Load a small image
We're using one with a resolution that is a multiple of 8, since we'd like to use 8x8 blocks. This [cryptopunk7804](https://www.larvalabs.com/cryptopunks/details/7804) is sourced from a [non-fungible token (NFT)](https://en.wikipedia.org/wiki/Non-fungible_token) that recently sold for $\sim\$7M$.

In [ ]:
I = plt.imread('../dip_pics/cryptopunk7804.png')
I = I[...,:3] # cut out the alpha channel
vis_hists(I)
print(arr_info(I))

## View as Blocks

You can see the above is a $24x24x3$ array of floats in $[0,1]$. [`view_as_blocks`](https://scikit-image.org/docs/dev/api/skimage.util.html?highlight=view_as_blocks#skimage.util.view_as_blocks) will allow us to consider this image as a grid of non-overlapping $8x8x3$ *blocks*.

In [ ]:
block_shape = (8,8,3)

view = view_as_blocks(I, block_shape)
arr_info(view)

As you can see, the resulting `view` variable is a $3x3$ grid of these $8x8x3$ blocks. Let's consider it as a linear list of blocks instead of a grid. When we reshape we have to be careful to maintain the $8x8x3$ elements of this list.

In [ ]:
block_view = np.reshape(view, [view.shape[0]*view.shape[1]] + list(block_shape))
arr_info(block_view)

## Montage

We can use [`montage`](https://scikit-image.org/docs/dev/api/skimage.util.html?highlight=view_as_blocks#skimage.util.montage) to reassemble the `block_view` into a single displayable image again. We'll add `padding` so that you can see the decomposition into blocks.

In [ ]:
I_block_grid = montage(block_view, grid_shape=view.shape[:2], multichannel=True, padding_width=1, fill=[1,1,1])

plt.figure(figsize=(5,5))
plt.imshow(I_block_grid)

## Transforming or Processing Blocks
With this `block_view` we can even even apply some kind of transform or function to each block, computing some block characteristics. Here, we'll replace each block with its mean color. We'll make a new list of blocks and then put our modified blocks into it, so that we can `montage` the modified image later.

In [ ]:
mod_block_view = np.zeros_like(block_view)

for i, block in enumerate(block_view):
    block_mean = block.mean(axis=(0,1)) # Get the average color in the block.
    
    mod_block_view[i] = block_mean

In [ ]:
mod_block_grid = montage(mod_block_view, grid_shape=view.shape[:2], multichannel=True, padding_width=1, fill=[1,1,1])

plt.figure(figsize=(5,5))
plt.imshow(mod_block_grid)

In this particular example we've processed a $24x24$ image into a $3x3$. With only 9 pixels of course, the image is pretty poorly resolved, but we have in a sense **compressed** the image 64-fold (a factor of 8 in $x$ and 8 in $y$).

## Try the above kind of decomposition in a larger image.

You have to be careful that the `block_shape` you choose to use is a factor of the shape of the image itself. Alternatively, you may need to [`resize`](https://scikit-image.org/docs/dev/api/skimage.transform.html?highlight=resize#skimage.transform.resize) the image in order to ensure that the blocks evenly divide the image.

## Keeping additional block information per block

In the above, every $8x8$ block is characterized by a single color, the average of the block. If you could keep any additional information about it, even at the cost of maybe being less compressive, what additional information might you keep?

Try this: try keeping four averages in each block. That is, consider the block as four quarters and store the mean of each quarter. This will be no doubt a better representation of the original, though at what cost?